In [2]:
# import relevant libraries
import pandas as pd
import json
import requests
import math
import time

url_sql = "https://api.opendota.com/api/explorer"

In [3]:
test = "SELECT * FROM heroes ORDER BY ID"

params = {"sql": test}

response = requests.get(url=url_sql, params=params)

print(response)

data = response.json()

df = pd.DataFrame(data["rows"])

df

<Response [200]>


,id,name,localized_name,primary_attr,attack_type,roles,legs
0,1,npc_dota_hero_antimage,Anti-Mage,agi,Melee,"[Carry, Escape, Nuker]",2
1,2,npc_dota_hero_axe,Axe,str,Melee,"[Initiator, Durable, Disabler, Carry]",2
2,3,npc_dota_hero_bane,Bane,all,Ranged,"[Support, Disabler, Nuker, Durable]",4
3,4,npc_dota_hero_bloodseeker,Bloodseeker,agi,Melee,"[Carry, Disabler, Nuker, Initiator]",2
4,5,npc_dota_hero_crystal_maiden,Crystal Maiden,int,Ranged,"[Support, Disabler, Nuker]",2
...,...,...,...,...,...,...,...
119,129,npc_dota_hero_mars,Mars,str,Melee,"[Carry, Initiator, Disabler, Durable]",2
120,135,npc_dota_hero_dawnbreaker,Dawnbreaker,str,Melee,"[Carry, Durable]",2
121,136,npc_dota_hero_marci,Marci,all,Melee,"[Support, Carry, Initiator, Disabler, Escape]",2
122,137,npc_dota_hero_primal_beast,Primal Beast,str,Melee,"[Initiator, Durable, Disabler]",2


In [5]:
df.to_csv("data/raw/heroes.csv", index=False, sep="\t")

In [8]:
project_id = "integral-cell-418310"



query ="""
SELECT *
FROM `integral-cell-418310.dota2.heroes`
"""

In [9]:
df_heroes = pd.read_gbq(query, project_id)

df_heroes.head()

/tmp/ipykernel_2118143/533210703.py:1: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  df_heroes = pd.read_gbq(query, project_id)


,id,name,localized_name,primary_attr,attack_type,roles,legs
0,1,npc_dota_hero_antimage,Anti-Mage,agi,Melee,"['Carry', 'Escape', 'Nuker']",2
1,4,npc_dota_hero_bloodseeker,Bloodseeker,agi,Melee,"['Carry', 'Disabler', 'Nuker', 'Initiator']",2
2,8,npc_dota_hero_juggernaut,Juggernaut,agi,Melee,"['Carry', 'Pusher', 'Escape']",2
3,12,npc_dota_hero_phantom_lancer,Phantom Lancer,agi,Melee,"['Carry', 'Escape', 'Pusher', 'Nuker']",2
4,32,npc_dota_hero_riki,Riki,agi,Melee,"['Carry', 'Escape', 'Disabler']",2


In [10]:
import ast

In [12]:
# Initialize an empty DataFrame to store the concatenated teamfights data
role_list = []
i = 0
# for every entry create entries in a new dataframe that will contain all picks/bans of that match id
for row in df_heroes.itertuples():
    # check if na:
    if pd.notna(row.roles):
        # Convert the value to a string and then replace single quotes with double quotes in the JSON string
        # teamfights_json = re.sub(r'False', 'false', teamfights_json)
        # teamfights_json = re.sub(r'True', 'true', teamfights_json)
        # print(teamfights_json)  # Print the modified JSON string before parsing
        list_from_string = ast.literal_eval(row.roles)
        for item in list_from_string:
            role_dict = {"role": item, "hero_id": row.id}
            # print(updated_item)
            role_list.append(role_dict)
    print(i)
    i += 1
# add the list of picks and bans to a df
df_roles = pd.DataFrame(role_list)
df_roles

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123


,role,hero_id
0,Carry,1
1,Escape,1
2,Nuker,1
3,Carry,4
4,Disabler,4
...,...,...
476,Nuker,138
477,Disabler,138
478,Carry,59
479,Durable,59


In [13]:
df_roles.to_gbq("dota2.roles", project_id=project_id)

/tmp/ipykernel_2118143/2195896658.py:1: FutureWarning: to_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.to_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.to_gbq
  df_roles.to_gbq("dota2.roles", project_id=project_id)
100%|██████████| 1/1 [00:00<00:00, 11366.68it/s]
